In [2]:
import pandas as pd
import zipfile
import geopandas as gpd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def pluto(mappluto):
    z = zipfile.ZipFile(mappluto)
    z.extractall('../data_initial/')
    
    boroughs = ['Staten_Island', 'Queens', 'Manhattan', 'Bronx', 'Brooklyn']
    abreevs = ['SI', 'QN', 'MN', 'BX', 'BK']

    pluto = []
    i = 0
    for elem in boroughs:
        temp = gpd.read_file('../data_initial/'+elem+'/'+abreevs[i]+'MapPLUTO.shp')
        temp['BBL'] = temp['BBL'].astype(str)
        #temp = temp.to_crs({'init':'epsg:4326'})
        pluto.append(temp)
        i+=1  

    global pluto_updated
    pluto_updated = pd.concat(pluto)

    return pluto_updated.head()

In [3]:
pluto('../data_initial/mappluto_16v2.zip')

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,None,437 PURDY AVENUE,R3-1,None,7,15120.0,23400.0,5007470028.0,1488,...,0,1960,None,10314,R3-1,None,None,None,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,0.0,None,CUNARD AVENUE,R3A/HS,None,4,21518.0,21518.0,5006230252.0,0,...,0,0,None,10304,R3A,None,None,None,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."
2,0.0,None,121 MOUNTAINVIEW AVENUE,R3-1,None,7,13561.0,22602.0,5007130017.0,1316,...,0,1920,None,10314,R3-1,None,None,None,21b,"POLYGON ((949278.1987999976 161346.9971999973,..."
3,0.0,None,26 FANNING STREET,R3X,None,7,9720.0,28740.0,5007140091.0,2188,...,0,1975,None,10314,R3X,None,None,None,21b,"POLYGON ((950061.0491999984 160815.3023999929,..."
4,0.0,None,335 SIMONSON AVENUE,R3A,None,7,7617.0,22118.0,5011840039.0,2392,...,0,1965,None,10303,R3A,None,None,None,20c,"POLYGON ((941981.4906000048 168120.2173999995,..."


In [161]:
pluto_updated.to_csv('../data_processed/pluto_16')

### there are no 'ownernames' with 'nyc housing authority' or 'nyc housing' so using ownertype instead

In [306]:
pluto_updated = pd.read_csv('../data_processed/pluto_16')

In [307]:
pluto_updated = pluto_updated.drop('Unnamed: 0',axis=1)

In [308]:
pluto_updated.head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,NaN,437 PURDY AVENUE,R3-1,NaN,7,15120.0,23400.0,5.007470e+09,1488,...,0,1960,NaN,10314,R3-1,NaN,NaN,NaN,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,0.0,NaN,CUNARD AVENUE,R3A/HS,NaN,4,21518.0,21518.0,5.006230e+09,0,...,0,0,NaN,10304,R3A,NaN,NaN,NaN,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."


In [309]:
t1 = pluto_updated[pluto_updated['UnitsRes'] != 0]
#t1 = t1[(t1['OwnerName'] != 'NYC HOUSING AUTHORITY') & (t1['OwnerName'] != 'NYC HOUSING')]

### create categorical variables from building class feature

In [310]:
var = []
i=0
for elem in t1['BldgClass']:
    if elem[i][0] == 'A':
        var.append(1)
    elif elem[i][0] == 'B':
        var.append(2)
    elif elem[i][0] == 'C':
        var.append(3)
    elif elem[i][0] == 'S':
        var.append(3)
    elif elem[i][0] == 'D':
        var.append(4)
    elif elem[i] == 'O8':
        var.append(4)
    elif elem[i][0] == 'L':
        var.append(6)
    elif elem[i][0] == 'R':
        var.append(5)
    else:
        var.append(0)

In [311]:
t1['bldgclass_updated'] = var

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [312]:
t1.head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry,bldgclass_updated
0,0.0,NaN,437 PURDY AVENUE,R3-1,NaN,7,15120.0,23400.0,5.007470e+09,1488,...,1960,NaN,10314,R3-1,NaN,NaN,NaN,20d,"POLYGON ((944681.4200000018 161477.381400004, ...",2
2,0.0,NaN,121 MOUNTAINVIEW AVENUE,R3-1,NaN,7,13561.0,22602.0,5.007130e+09,1316,...,1920,NaN,10314,R3-1,NaN,NaN,NaN,21b,"POLYGON ((949278.1987999976 161346.9971999973,...",1


In [313]:
len(t1)

759588

In [314]:
t2 = pluto_updated[pluto_updated['ResArea'] != 0]
#t2 = t2[(t2['OwnerType'] != 'O') & (t2['OwnerType'] != 'C')]

In [315]:
len(t2)

758493

### selecting to use 'unitsres' as the value for not equalling zero because there are more rows included

In [316]:
t1.columns

Index([u'APPBBL', u'APPDate', u'Address', u'AllZoning1', u'AllZoning2',
       u'AreaSource', u'AssessLand', u'AssessTot', u'BBL', u'BldgArea',
       u'BldgClass', u'BldgDepth', u'BldgFront', u'Block', u'BoroCode',
       u'Borough', u'BsmtCode', u'BuiltCode', u'BuiltFAR', u'CB2010', u'CD',
       u'CT2010', u'ComArea', u'CommFAR', u'CondoNo', u'Council', u'EDesigNum',
       u'Easements', u'ExemptLand', u'ExemptTot', u'Ext', u'FacilFAR',
       u'FactryArea', u'FireComp', u'GarageArea', u'HealthArea', u'HistDist',
       u'IrrLotCode', u'LandUse', u'Landmark', u'Lot', u'LotArea', u'LotDepth',
       u'LotFront', u'LotType', u'LtdHeight', u'MAPPLUTO_F', u'NumBldgs',
       u'NumFloors', u'OfficeArea', u'OtherArea', u'Overlay1', u'Overlay2',
       u'OwnerName', u'OwnerType', u'PLUTOMapID', u'PolicePrct', u'ProxCode',
       u'ResArea', u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Leng',
       u'SPDist1', u'SPDist2', u'Sanborn', u'SchoolDist', u'SplitZone',
       u'StrgeArea', 

In [317]:
t1 = t1.drop(['APPBBL', 'APPDate', 'Address', 'AreaSource', 'BldgDepth', 'BldgFront', \
                                   'Block', 'BoroCode', 'Borough', 'CB2010', 'CD', 'CT2010', 'ComArea', \
                                   'CommFAR', 'CondoNo', 'Council', 'EDesigNum', 'FacilFAR', 'FactryArea', \
                                   'FireComp', 'GarageArea', 'HealthArea', 'HistDist', 'IrrLotCode', 'Landmark', \
                                   'Lot', 'LotArea', 'LotDepth', 'LotFront', 'LtdHeight', 'MAPPLUTO_F', \
             'OfficeArea', 'OtherArea', 'Overlay1', 'Overlay2', 'PLUTOMapID', 'PolicePrct', 'ResidFAR', \
             'RetailArea', 'Sanborn', 'SchoolDist', 'StrgeArea', 'TaxMap', 'Tract2010', 'YearAlter1', \
             'YearAlter2', 'ZoneDist1', 'ZipCode', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'ZoneMap', \
             'XCoord', 'Version', 'YCoord', 'ZMCode', 'SPDist1', 'SPDist2', 'ExemptTot', 'BuiltCode', \
             'AllZoning1', 'AllZoning2', 'ExemptLand', 'geometry', 'Easements', 'SHAPE_Area', 'SHAPE_Leng', \
             'OwnerName', 'Ext', 'SplitZone', 'BldgClass', 'AssessTot', 'BldgArea', 'UnitsTotal'], axis=1)

In [318]:
t1.columns

Index([u'AssessLand', u'BBL', u'BsmtCode', u'BuiltFAR', u'LandUse', u'LotType',
       u'NumBldgs', u'NumFloors', u'OwnerType', u'ProxCode', u'ResArea',
       u'UnitsRes', u'YearBuilt', u'bldgclass_updated'],
      dtype='object')

### explanation of variables in dataset
* assessland -  tentative assessed land value for Fiscal Year 2017
* assesstot - tentative assessed total value for Fiscal Year 2017
* bldgarea - total gross area in square feet
* bldgclass - code describing the major use of structures on the tax lot
* bsmtcode - code describing the basement type/grade - 0: no basement, 1: full basement above grade, 2: full basement below grade, 3: partial basement above grade, 4: partial basement below grade, 5: unknown
* landuse - code for the tax lot's land use category (01-09)
* lottype - code indicating the location of the tax lot to another tax lot and/or the water (1-9)
* numbldgs - number of buildings in the tax lot
* numfloors - the tallest building on the tax lot, the number of full and partial stories starting from the ground floor
* ownertype - M: mix city and private ownership, P: private ownership, X: mixed (excludes C, M, O, or P ownerships), blank: unknown
* proxcode - physical relationship of the building to neighboring buildings - 0: not available, 1: detatched, 2: semi-attached, 3: attached
* resarea - estimate of the exterior dimensions of the portion of the structure(s) allocated for residential use
* unitsres - sum of residential units in all buildings on the tax lot
* unitstotal - sum of residential and non-residential (offices, retail stores, etc.) units in all buildings on the tax lot
* yearbuilt - year structure was built

### merge pluto with complaint data

In [319]:
comp = pd.read_csv('../data_processed/311_heathotwater_2014_2016_BBL_WEEK_AVG.csv')
comp.head()

,BBL,com_count,BBL.1
0,1000047501,0.020408,1000047501
1,1000100032,0.006803,1000100032
2,1000150022,0.006803,1000150022
3,1000160015,0.006803,1000160015
4,1000160020,0.013605,1000160020


In [320]:
comp = comp.drop('BBL.1',axis=1)

In [321]:
print comp['BBL'].dtypes
print t1['BBL'].dtypes

int64
float64


In [322]:
import numpy as np

In [323]:
t1['BBL'] = pd.to_numeric(t1['BBL'])

In [324]:
bbl = np.array(t1['BBL'])
bbl = bbl.astype(int)

In [325]:
t1['BBL'] = bbl

In [326]:
print comp['BBL'].dtypes
print t1['BBL'].dtypes

int64
int64


In [327]:
features = pd.merge(t1, comp, on ='BBL')

In [328]:
print len(t1)
print len(comp)
print len(features)

759588
53310
51782


In [329]:
features.head()

,AssessLand,BBL,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,OwnerType,ProxCode,ResArea,UnitsRes,YearBuilt,bldgclass_updated,com_count
0,8441.0,5010310021,2.0,0.70,2.0,5.0,1,2.75,NaN,1.0,4710,3,1920,3,0.013605
1,6480.0,5012460037,2.0,0.92,1.0,5.0,1,2.00,NaN,1.0,2700,2,2003,2,0.034014
2,9007.0,5033390088,2.0,0.48,1.0,5.0,1,2.00,NaN,1.0,1976,2,1955,2,0.054422
3,3980.0,5000810024,2.0,0.54,1.0,5.0,1,2.00,NaN,1.0,1344,1,1901,1,0.074830
4,96750.0,5000440001,5.0,0.66,2.0,3.0,1,2.00,P,0.0,14156,16,1984,3,0.006803


In [330]:
features.to_csv('../data_processed/mappluto_16_features')

### select features with ensemble decision tree and random forest classifiers

In [331]:
features = features.drop('BBL',axis=1)

In [332]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [333]:
import random
random.seed(123)

In [334]:
le = preprocessing.LabelEncoder()

In [335]:
#le.fit([elem for elem in features['bldgclass_updated'].unique()])
#features['bc_updated'] = le.transform([elem for elem in features['bldgclass_updated']])
le.fit([elem for elem in features['OwnerType'].unique()])
features['ownertype_updated'] = le.transform([elem for elem in features['OwnerType']])

In [336]:
features = features.drop('OwnerType',axis=1)

In [337]:
features = features.apply(pd.to_numeric)
features = features.astype(int)

In [338]:
features.head()

,AssessLand,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,ResArea,UnitsRes,YearBuilt,bldgclass_updated,com_count,ownertype_updated
0,8441,2,0,2,5,1,2,1,4710,3,1920,3,0,5
1,6480,2,0,1,5,1,2,1,2700,2,2003,2,0,5
2,9007,2,0,1,5,1,2,1,1976,2,1955,2,0,5
3,3980,2,0,1,5,1,2,1,1344,1,1901,1,0,5
4,96750,5,0,2,3,1,2,0,14156,16,1984,3,0,3


In [339]:
features['BBL'] = comp['BBL']

In [340]:
features.head(2)

,AssessLand,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,ResArea,UnitsRes,YearBuilt,bldgclass_updated,com_count,ownertype_updated,BBL
0,8441,2,0,2,5,1,2,1,4710,3,1920,3,0,5,1000047501
1,6480,2,0,1,5,1,2,1,2700,2,2003,2,0,5,1000100032


In [341]:
features.to_csv('../data_processed/mappluto_16_features_categorical')

In [344]:
features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]].head(2)

,AssessLand,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,ResArea,UnitsRes,YearBuilt,bldgclass_updated,ownertype_updated
0,8441,2,0,2,5,1,2,1,4710,3,1920,3,5
1,6480,2,0,1,5,1,2,1,2700,2,2003,2,5


In [345]:
features.ix[:,-3].head(2)

0    0
1    0
Name: com_count, dtype: int64

In [346]:
model = ExtraTreesClassifier()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.20120783  0.01823439  0.08186167  0.01875477  0.02798844  0.01722529
  0.04156943  0.01090124  0.21431433  0.18595941  0.154979    0.00893061
  0.0180736 ]


In [347]:
etc = pd.DataFrame()
etc['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]].columns
etc['etc_value'] = model.feature_importances_

In [348]:
etc = etc.sort_values('etc_value',ascending=False)
etc

,feature_name,etc_value
8,ResArea,0.214314
0,AssessLand,0.201208
9,UnitsRes,0.185959
10,YearBuilt,0.154979
2,BuiltFAR,0.081862
6,NumFloors,0.041569
4,LotType,0.027988
3,LandUse,0.018755
1,BsmtCode,0.018234
12,ownertype_updated,0.018074


In [349]:
model = ExtraTreesRegressor()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.2408583   0.01476371  0.07176503  0.02379969  0.01092413  0.01600142
  0.01947615  0.00776669  0.2546169   0.19682396  0.10921983  0.00327883
  0.03070535]


In [350]:
etr = pd.DataFrame()
etr['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]].columns
etr['etr_value'] = model.feature_importances_

In [351]:
etr = etr.sort_values('etr_value',ascending=False)
etr

,feature_name,etr_value
8,ResArea,0.254617
0,AssessLand,0.240858
9,UnitsRes,0.196824
10,YearBuilt,0.109220
2,BuiltFAR,0.071765
12,ownertype_updated,0.030705
3,LandUse,0.023800
6,NumFloors,0.019476
5,NumBldgs,0.016001
1,BsmtCode,0.014764


In [352]:
model = RandomForestClassifier()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.22382016  0.0237243   0.05061796  0.02327499  0.03068092  0.01153825
  0.03372831  0.02129329  0.2521057   0.16504263  0.12863069  0.00973632
  0.02580647]


In [353]:
rfc = pd.DataFrame()
rfc['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]].columns
rfc['rfc_value'] = model.feature_importances_

In [354]:
rfc = rfc.sort_values('rfc_value',ascending=False)
rfc

,feature_name,rfc_value
8,ResArea,0.252106
0,AssessLand,0.223820
9,UnitsRes,0.165043
10,YearBuilt,0.128631
2,BuiltFAR,0.050618
6,NumFloors,0.033728
4,LotType,0.030681
12,ownertype_updated,0.025806
1,BsmtCode,0.023724
3,LandUse,0.023275


In [355]:
model = RandomForestRegressor()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.26246676  0.0165231   0.01950815  0.05296956  0.00642113  0.00695991
  0.0172565   0.05817106  0.32756883  0.12781165  0.08103133  0.00710784
  0.01620416]


In [356]:
rfr = pd.DataFrame()
rfr['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]].columns
rfr['rfr_value'] = model.feature_importances_

In [357]:
rfr = rfr.sort_values('rfr_value',ascending=False)
rfr

,feature_name,rfr_value
8,ResArea,0.327569
0,AssessLand,0.262467
9,UnitsRes,0.127812
10,YearBuilt,0.081031
7,ProxCode,0.058171
3,LandUse,0.052970
2,BuiltFAR,0.019508
6,NumFloors,0.017257
1,BsmtCode,0.016523
12,ownertype_updated,0.016204


In [358]:
merged = pd.merge(etc, rfc, on='feature_name')

In [359]:
merged = pd.merge(merged, rfr, on='feature_name')

In [360]:
merged = pd.merge(merged, etr, on ='feature_name')

In [361]:
merged['avg'] = merged.mean(axis=1)

In [362]:
merged = merged.sort_values('avg', ascending=False)

In [363]:
merged

,feature_name,etc_value,rfc_value,rfr_value,etr_value,avg
0,ResArea,0.214314,0.252106,0.327569,0.254617,0.262151
1,AssessLand,0.201208,0.223820,0.262467,0.240858,0.232088
2,UnitsRes,0.185959,0.165043,0.127812,0.196824,0.168909
3,YearBuilt,0.154979,0.128631,0.081031,0.109220,0.118465
4,BuiltFAR,0.081862,0.050618,0.019508,0.071765,0.055938
7,LandUse,0.018755,0.023275,0.052970,0.023800,0.029700
5,NumFloors,0.041569,0.033728,0.017257,0.019476,0.028008
11,ProxCode,0.010901,0.021293,0.058171,0.007767,0.024533
9,ownertype_updated,0.018074,0.025806,0.016204,0.030705,0.022697
6,LotType,0.027988,0.030681,0.006421,0.010924,0.019004


In [364]:
features['com_count'].unique()

array([ 0,  1,  7,  3, 17,  2,  5, 30, 20,  9,  4,  6, 12, 10,  8])